In [1]:
import modelscope
from transformers import AutoTokenizer,AutoModelForCausalLM
import json
import torch

/usr/local/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('./muice-dataset-train.catgirl (2).json','r') as f:
    data = json.load(f)

In [3]:
max_len = 0
for d in data:
    max_len = max(len(d['instruction']),max_len)
    max_len = max(len(d['output']),max_len)
max_len

146

In [4]:
max_length = 256

In [5]:
model = AutoModelForCausalLM.from_pretrained("./DeepSeek")
tokenizer = AutoTokenizer.from_pretrained("./DeepSeek")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [6]:
# def token(example,max_length=180):
#     instruction = example['instruction']
#     output = example['output']
#     prompt = [{"role":"user","content":instruction}]
#     prompt_token = tokenizer.apply_chat_template(prompt,tokenize=False)
#     q = tokenizer.encode(prompt_token)
#     a = tokenizer.encode(output)
#     input_ids = q+a
#     labels = a
#     attention_mask = [-100]*len(q) + [1]*len(a)
#     if len(input_ids)<=max_length:
#         padding_length = max_length-len(input_ids)
#         input_ids = [-100]*padding_length+ input_ids
#         labels = [-100]*(padding_length+len(q)) +labels
#         attention_mask = [0]*padding_length+attention_mask
#     else:
#         input_ids = input_ids[:max_length]
#         labels = labels[:max_length]
#         attention_mask = attention_mask[:max_length]
#     return {
#         "input_ids":input_ids,
#         "labels":labels,
#         "attention_mask":attention_mask
#     }
# def token(example, max_length=180):
#     instruction = example['instruction']
#     output = example['output']
#     prompt = [{"role": "user", "content": example["instruction"]}]
#     prompt_token = tokenizer.apply_chat_template(prompt, tokenize=False)
#     tokenized = tokenizer(
#         prompt_token,
#         text_target=example["output"],
#         max_length=max_length,
#         padding="max_length",
#         truncation=True,
#         return_tensors="pt",
#     )
#     input_ids = tokenized["input_ids"][0]
#     labels = tokenized["labels"][0]
#     attention_mask = tokenized["attention_mask"][0]
  
    
#     if len(input_ids) < max_length:
#         pad_len = max_length - len(input_ids)
#         input_ids = [tokenizer.pad_token_id] * pad_len + input_ids
#         attention_mask = [0] * pad_len + attention_mask
#         labels = [-100] * pad_len + labels
#     else:
#         input_ids = input_ids[:max_length]
#         attention_mask = attention_mask[:max_length]
#         labels = labels[:max_length]
  # prompt = [{"role": "user", "content": instruction}]
    # prompt_token = tokenizer.apply_chat_template(prompt, tokenize=False)
    
    # q = tokenizer.encode(prompt_token, add_special_tokens=False)
    # a = tokenizer.encode(output, add_special_tokens=False)
    
    # input_ids = q + a
    # attention_mask = [-100] * len(q) + [1] * len(a)
    # labels = [-100] * len(q) + a
def token(example, max_length=180):
    instruction = example['instruction']
    output = example['output']
    prompt = [{"role": "user", "content": example["instruction"]}]
    q = tokenizer.apply_chat_template(prompt, tokenize=False,add_generation_prompt=True)
    
    q_input_ids = tokenizer.encode(q)
    a_input_ids = tokenizer.encode(output)
    input_ids = q_input_ids+a_input_ids
    attention_mask = [1]*len(input_ids)
    labels = [-100]*len(q_input_ids)+a_input_ids
    if len(input_ids) < max_length:
        pad_len = max_length - len(input_ids)
        input_ids = [tokenizer.pad_token_id] * pad_len + input_ids
        attention_mask = [0] * pad_len + attention_mask
        labels = [-100] * pad_len + labels
    else:
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    
    # Sanity check
    assert len(input_ids) == max_length
    assert len(labels) == max_length
    assert len(attention_mask) == max_length

    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask
    }

data_ids = []
for d in data:
    id_dict = token(d)
    data_ids.append(id_dict)
    

In [7]:
from torch.utils.data import Dataset
class MyData(Dataset):
    def __init__(self,data):
        super().__init__()
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        data = self.data[index]
        
        return {
            "input_ids": torch.tensor(data["input_ids"], dtype=torch.long),
            "labels": torch.tensor(data["labels"], dtype=torch.long),
            "attention_mask": torch.tensor(data["attention_mask"], dtype=torch.long)
        }

In [8]:
# data_ids[:5]['input_ids']

In [9]:
from torch.utils.data import DataLoader
my_data = MyData(data_ids)
train_dataloader = DataLoader(my_data, shuffle=True, batch_size=8)


In [10]:
tokenizer.pad_token_id

151643

In [11]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-6)
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for i,batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        
        print(loss.item())

  0%|          | 1/483 [00:45<6:03:02, 45.19s/it]

7.780388832092285


  0%|          | 2/483 [01:41<6:52:40, 51.48s/it]

6.4830708503723145


  1%|          | 3/483 [02:08<5:24:55, 40.62s/it]

...

  2%|▏         | 10/483 [05:24<3:42:09, 28.18s/it]

3.899470090866089


  2%|▏         | 11/483 [05:58<3:55:43, 29.97s/it]

4.092857837677002


KeyboardInterrupt: 

In [16]:
# 编码输入
input = tokenizer("沐雪的功能是什么？", return_tensors="pt")  # 加上 return_tensors="pt" 以得到 PyTorch 张量格式

# 生成回复
output_ids = model.generate(**input, max_new_tokens=50)  # 可加参数控制生成长度等

# 解码输出
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [17]:
response

'沐雪的功能是什么？我之前学过一些关于雪的知识，但对沐雪的了解还不是很透彻。请问沐雪的主要功能是什么？它有什么优点和缺点？我之前学过一些关于雪的知识，但对沐雪的了解还不是很'

In [18]:
# 保存模型和 tokenizer
save_path = "./my_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('./my_model/tokenizer_config.json',
 './my_model/special_tokens_map.json',
 './my_model/tokenizer.json')

In [19]:
# 读取预训练的模型和tokenizer
model = AutoModelForCausalLM.from_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained(save_path)


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 19.06it/s]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw